In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    when, col, substring, expr,
    to_date, year, sum, count,
    lit, regexp_replace, lpad, length
)


spark = SparkSession.builder.appName("sim-salvador") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .config("spark.sql.repl.eagerEval.enabled", "true") \
    .config("spark.sql.repl.eagerEval.maxNumRows", 20) \
    .getOrCreate()
df = spark.read.csv(
    "curso-analise-de-dados/dados/sim_salvador_2023.csv",
    header=True,
    inferSchema=True
)

df = df.withColumn(
    "GENERO",
    when(col("SEXO")   == 1, "Masculino")
     .when(col("SEXO") == 2, "Feminino")
     .when(col("SEXO") == 0, "Ignorado")
     .otherwise(None)  # NA_character_
)  # when/otherwise é o análogo ao case_when.[web:54]
df.groupBy("SEXO").count().orderBy("count", ascending=False).show()
df.groupBy("GENERO").count().orderBy("count", ascending=False).show()
df.show(5)
df.printSchema()

25/12/30 20:03:12 WARN Utils: Your hostname, mauro-lucio resolves to a loopback address: 127.0.1.1; using 192.168.0.104 instead (on interface enp1s0)
25/12/30 20:03:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/30 20:03:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+----+-----+
|SEXO|count|
+----+-----+
|   1|  150|
|   2|  111|
+----+-----+

+---------+-----+
|   GENERO|count|
+---------+-----+
|Masculino|  150|
| Feminino|  111|
+---------+-----+

+---------+--------+-----+----+-------+--------+--------+---------+
|CODMUNRES| DTOBITO|IDADE|SEXO|RACACOR|CAUSABAS|TIPOBITO|   GENERO|
+---------+--------+-----+----+-------+--------+--------+---------+
|   292720| 2012023|  457|   2|      4|    J980|       2| Feminino|
|   292720| 5012023|  482|   2|      2|    I500|       2| Feminino|
|   292720| 7012023|  452|   2|      4|    C220|       2| Feminino|
|   292720| 8012023|  456|   2|      4|    W789|       2| Feminino|
|   292720|10012023|  468|   1|      4|    X954|       2|Masculino|
+---------+--------+-----+----+-------+--------+--------+---------+
only showing top 5 rows

root
 |-- CODMUNRES: integer (nullable = true)
 |-- DTOBITO: integer (nullable = true)
 |-- IDADE: integer (nullable = true)
 |-- SEXO: integer (nullable = true)
 |-- RACACOR:

In [2]:
# extrair tipo_idade e idade (str_sub equivalente)
df = df.withColumn("tipo_idade", substring(col("IDADE").cast("string"), 1, 1)) \
        .withColumn("idade", substring(col("IDADE").cast("string"), 2, 100))
df.select("IDADE", "tipo_idade", "idade").show(10, truncate=False)

# idade_anos com case_when
df = df.withColumn(
    "idade_anos",
    when(col("tipo_idade").cast("int") <= 3, 0)
    .when(col("tipo_idade") == "4", col("idade").cast("int"))
    .when(col("tipo_idade") == "5", (100 + col("idade").cast("int")))
    .otherwise(None)
)
df.select("IDADE", "tipo_idade", "idade", "idade_anos").show(10)

# DTOBITO com date e ano_obito
df = df.withColumn(
    "DTOBITO_clean", 
    lpad(col("DTOBITO").cast("string"), 8, "0")
).withColumn(
    "DTOBITO_dt", 
    to_date(col("DTOBITO_clean"), "ddMMyyyy")
).withColumn(
    "ano_obito", 
    year(col("DTOBITO_dt"))
)

df.select("DTOBITO", "DTOBITO_clean", "DTOBITO_dt", "ano_obito").show(10, truncate=False)

# contagens finais
df.groupBy("ano_obito").count().orderBy("ano_obito").show()
df.groupBy("GENERO", "ano_obito").count().orderBy("ano_obito", "GENERO").show()

# Schema (equivalente a glimpse)
df.printSchema()
df.describe().show()

+-----+----------+-----+
|IDADE|tipo_idade|idade|
+-----+----------+-----+
|57   |4         |57   |
|82   |4         |82   |
|52   |4         |52   |
|56   |4         |56   |
|68   |4         |68   |
|39   |4         |39   |
|89   |4         |89   |
|81   |4         |81   |
|71   |4         |71   |
|83   |4         |83   |
+-----+----------+-----+
only showing top 10 rows

+-----+----------+-----+----------+
|IDADE|tipo_idade|idade|idade_anos|
+-----+----------+-----+----------+
|   57|         4|   57|        57|
|   82|         4|   82|        82|
|   52|         4|   52|        52|
|   56|         4|   56|        56|
|   68|         4|   68|        68|
|   39|         4|   39|        39|
|   89|         4|   89|        89|
|   81|         4|   81|        81|
|   71|         4|   71|        71|
|   83|         4|   83|        83|
+-----+----------+-----+----------+
only showing top 10 rows

+--------+-------------+----------+---------+
|DTOBITO |DTOBITO_clean|DTOBITO_dt|ano_obito|
+-

25/12/30 20:03:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------+--------------------+------------------+-------------------+------------------+--------+--------+---------+-------------------+------------------+--------------------+---------+
|summary|CODMUNRES|             DTOBITO|             idade|               SEXO|           RACACOR|CAUSABAS|TIPOBITO|   GENERO|         tipo_idade|        idade_anos|       DTOBITO_clean|ano_obito|
+-------+---------+--------------------+------------------+-------------------+------------------+--------+--------+---------+-------------------+------------------+--------------------+---------+
|  count|      261|                 261|               261|                261|               261|     261|     261|      261|                261|               261|                 261|      261|
|   mean| 292720.0|1.6828153268199235E7| 66.03065134099617| 1.4252873563218391|  3.21900826446281|    NULL|     2.0|     NULL| 3.9731800766283527| 66.70114942528735|1.6828153268199235E7|   2023.0|
| stddev|      